In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_excel(r'C:\Users\sanla\Desktop\Prediction of CO Conversion\Dataset_Intern.xlsx')
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [4]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

In [5]:
from xgboost import XGBRegressor
regressor = XGBRegressor()
regressor.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [6]:
y_pred = regressor.predict(x_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[68.73 89.8 ]
 [60.11 99.4 ]
 [46.18 24.2 ]
 ...
 [ 6.12 20.6 ]
 [79.83  4.71]
 [22.58 22.4 ]]


In [7]:
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
score = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(score)
print(mse)

0.5676853088898399
466.4123047910386


In [8]:
from sklearn.model_selection import cross_val_score
scorer = make_scorer(mean_squared_error)
scores = cross_val_score(estimator = regressor, X = x_train, y = y_train, scoring = 'r2', cv = 10)
amse = cross_val_score(estimator = regressor, X = x_train, y = y_train, scoring = scorer, cv = 10)
print(scores.mean() * 100)
print(scores.std() * 100)
print(amse.mean())

81.17381941132713
3.987019831337587
194.9293011196362


In [9]:
from sklearn.model_selection import GridSearchCV
parameters = [{
    'learning_rate': [0.1],
    'n_estimators': [326],
    'max_depth': [None],
    'min_child_weight': [5],
    'subsample': [0.9],
    'colsample_bytree': [0.8],
    'gamma': [0.125],
    'reg_alpha': [0.1],
    'reg_lambda': [0.1]
}]
grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parameters,
                           scoring = 'r2',
                          cv = 15,
                          n_jobs = -1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=15,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None,...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid=[{'colsample_bytree': [0.8], 'gamma': [0.125],
                          'learning_rate': [0.1], 'max_depth': [None],
                          'min_child_weight': [5], 'n_estimators': [326],
                          'reg_alpha': [0.1], 'reg_lambda': [0.1],
                          'subsample': [0.9]}],
             scoring='r2')

In [10]:
print(grid_search.best_score_)

0.825526897649975


In [11]:
print(grid_search.best_params_)

{'colsample_bytree': 0.8, 'gamma': 0.125, 'learning_rate': 0.1, 'max_depth': None, 'min_child_weight': 5, 'n_estimators': 326, 'reg_alpha': 0.1, 'reg_lambda': 0.1, 'subsample': 0.9}


In [12]:
amse = cross_val_score(estimator = grid_search, X = x_train, y = y_train, scoring = scorer, cv = 15)
print(amse.mean())

180.02909921749037
